<a href="https://colab.research.google.com/github/matthewpecsok/data_engineering/blob/main/de_api_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install required libraries

In [1]:
!pip install flask # webserver
!pip install faker # fake data generator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.3 MB/s eta 0:00:00


In [2]:
import requests

# create the API app in flask

use multithreading to run Flask in a new thread so the notebook is free to continue executing other code.

In [3]:
from flask import Flask, request, jsonify
import sqlite3
import threading
import multiprocessing, time

app = Flask(__name__)

DATABASE = 'customers.db'

@app.route("/")
def home():
    return "Hello World! This is the API homepage. No swagger."


def create_table():
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT NOT NULL,
            phone TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

@app.route('/customers', methods=['GET'])
def get_customers():
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM customers")
    customers = cursor.fetchall()
    conn.close()
    return jsonify(customers)

@app.route('/customers/<int:customer_id>', methods=['GET'])
def get_customer_by_id(customer_id):
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM customers where id = ?",(customer_id,))
    customers = cursor.fetchall()
    conn.close()
    return jsonify(customers)

@app.route('/customers', methods=['POST'])
def add_customer():
    name = request.json.get('name')
    email = request.json.get('email')
    phone = request.json.get('phone')

    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO customers (name, email, phone) VALUES (?, ?, ?)", (name, email, phone))
    conn.commit()
    conn.close()

    return jsonify({'message': 'Customer added successfully'})

@app.route('/customers/<int:customer_id>', methods=['PUT'])
def update_customer(customer_id):
    name = request.json.get('name')
    email = request.json.get('email')
    phone = request.json.get('phone')

    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("UPDATE customers SET name=?, email=?, phone=? WHERE id=?", (name, email, phone, customer_id))
    conn.commit()
    conn.close()

    return jsonify({'message': 'Customer updated successfully'})

@app.route('/customers/<int:customer_id>', methods=['DELETE'])
def delete_customer(customer_id):
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM customers WHERE id=?", (customer_id,))
    conn.commit()
    conn.close()

    return jsonify({'message': 'Customer deleted successfully'})

if __name__ == '__main__':
    create_table()
    multiprocessing.Process(target=app.run).start() # this is required for background running so the cell can be released.



 * Serving Flask app '__main__'


# get public url

use the url here to make sure flask is running. you should get a webpage that says '*Hello World! This is the API homepage.*'

In [4]:
# this cell outputs the url you can use in your browser to see if the website/api is running.

from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://01kvgzg464uo-496ff2e9c6d22116-5000-colab.googleusercontent.com/


## function: get customers

In [5]:
def helper_get_customers():

  import requests

  url = f'http://localhost:5000/customers'

  response = requests.get(url)
  customers = response.json()

  if response.status_code == 200:
      return customers
  else:
      return(f'Error: {response.status_code} - {response.text}')

  return customers


def helper_get_customer(customerid):

  import requests

  url = f'http://localhost:5000/customers/{customerid}'

  response = requests.get(url)


  if response.status_code == 200:
      customer = response.json()
      return customer
  else:
      return(f'Error: {response.status_code} - {response.text}')

  return customer


# function: generate fake customer

generate fake customer, post to the API which inserts the record into sqlite.

In [6]:
def generate_customers():

  import requests
  from faker import Faker

  url = 'http://localhost:5000/customers'

  faker = Faker()

  new_customer = {
      'name': faker.name(),
      'email': faker.email(),
      'phone': faker.phone_number()
  }

  response = requests.post(url, json=new_customer)

  if response.status_code == 200:
      print('New customer added successfully.')
  else:
      print(f'Error: {response.status_code} - {response.text}')


# generate a new customer record and post it to the API

In [7]:
generate_customers()

New customer added successfully.


# get all customer records with the API

In [8]:
helper_get_customers()

[[1, 'Thomas Torres', 'pittsgary@example.net', '491-357-9338x6362']]

In [9]:
helper_get_customer(1)

[[1, 'Thomas Torres', 'pittsgary@example.net', '491-357-9338x6362']]

# query the database with pandas

In [10]:
import pandas as pd

In [11]:
DATABASE = 'customers.db'

In [12]:
conn = sqlite3.connect(DATABASE)

In [13]:
pd.read_sql_query("select * from customers",conn)

,id,name,email,phone
0,1,Thomas Torres,pittsgary@example.net,491-357-9338x6362


## kill the webserver


In [14]:
!lsof -i :5000

COMMAND PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
python3 368 root   45u  IPv4  24755      0t0  TCP localhost:5000 (LISTEN)


In [15]:
!kill -9 1607

/bin/bash: line 1: kill: (1607) - No such process
